# House price prediction data set

Consisting of 4600 entries for homes in different US cities, this dataset includes 18 different variables that describe various property details such as size and location characteristics. 

**Columns:**

date: Date the house was sold<br>
price: Sale price of the house (Prediction target)<br>
bedrooms: Number of the bedrooms<br>
bathrooms: Number the bathrooms<br>
sqft_living: Square footage of the home<br>
sqft_lot: Square footage of the lot<br>
floors: Total floors (levels) in the house<br>
waterfront: House which has a view to a waterfront<br>
view: Number of views<br>
condition: How good the condition is overall<br>
sqft_above: Square footage of the house apart from basement<br>
sqft_basement: Square footage of the basement<br>
yr_built: Building year of the house<br>
yr_renovated: Renovation year of the house<br>
street: Street where the house is located<br>
city: City where the house is located<br>
statezip: Zip code of the region where the house is located<br>
country: Country where the house is located (USA)<br>


There are 8 categorical (half nominal and half ordinal depending on the measurement level) and 9 discrete variables.

Our ordinal variables, waterfront, view and and renovation year, appear numerically, but we will evaluate them in two categories, 0 and 1, depending on whether they are in the relevant house or not. Also, the condition has 5 unique values that increase from 1 to 5 based on the state of the house. And as you might guess, street, city, statezip, and country are the nominal ones.

**Types of Data**
1. Categorical (gender, blood type, aswers of yes/no questions, satisfaction/condition level etc.)
2. Numerical<br>
a. Discrete (finite numbers- # of children/customer/room, physical money, time on a clock etc.)<br>
b. Continuous (infinite numbers- measurements(weight, height), time in general etc.)<br>

**Levels of Measurement**
1. Qualitative<br>
a. Nominal (Can not be ordered- gender, blood type, seasons)<br>
b. Ordinal (Takes values with an order or rank- satisfaction, condition level)<br>
2. Quantitative<br>
a. Interval (Don't have a true zero- temperature, pH, IQ test etc.)<br>
b. Ratios (Have a true zero- temperature only in Kelvin, amount, distance, pulse etc.)<br>



# Importing Relevant Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import math

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import statsmodels.api as sm

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
import sklearn.metrics as metrics

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Loading and Exploring the data set

In [ ]:
raw_data = pd.read_csv("../input/housedata/data.csv")

In [ ]:
# Quick look to the data

raw_data.head()

In [ ]:
# Checking for missing values and data types of each column

raw_data.info()

In [ ]:
# Added "include="all"" parameter to see all columns. (to see not only numeric ones)
# Based on the unique value count, city can be used by converting to dummies

raw_data.describe(include='all').T

In [ ]:
# Let's check the accuracy of Linear regression without any data preprocessing

# Define the variables
x1 = raw_data.drop(['price', 'date', 'street', 'city', 'statezip', 'country'], axis = 1) 
y = raw_data['price']

# Add a constant. Esentially, we are adding a new column (equal in lenght to x), which consists only of 1's
x = sm.add_constant(x1)
# Fit the model, according to the OLS (ordinary least squares) method with a dependent variable y and an independent x
results = sm.OLS(y,x).fit()
# Print a summary of the model
results.summary()

## Interpreting the Regression Table

I used OLS (Ordinary least squares) model from statsmodel, which is often used in linear regression analysis. It compares the difference between individual points in our data set and the predicted best fit line to measure the amount of error produced.

In order to interpret this summary and to get an overview of the statistics here, we need to know the components in the table. I will try to explain the most important ones in my opinion.

Typically when using statsmodels we have three main tables. The table called the model summary, which is at the top of the summary, contains information that is mostly known to us; such as **dependent variable, model, method, date and time we created the model, number of observations.

**DF Residuals** is widely used in many contexts throughout statistics, including hypothesis testing, probability distributions, and regression analysis. It is represents the degrees of freedom of our model, the formula is given below.
'(number of observations) - (number of predicting variables - 1)'<br>
'(n-k-1 = 4600-11-1 = 4588)'. <br>


**R-squared** is one of the most important explanatory power, a measure of how much of the independent variable is explained by changes in our dependent variables. R-squared 0 means our regression explains NONE of the variabilty of the data, while R-sguared equal 1 means it explains the entire variability. In our case, the model can only explain 21% of our dataset.

Checking the **adjusted R-squared** is essential to analyze the effectiveness of multiple dependent variables on the model. Excessive use of variables is penalized when measuring, so while adding some variables that don't contribute to our model can increase our R-square, it often has a negative impact on the adjusted R-squared. It is always smaller than R squared.

In the second part of our summary, the table of coefficients, I want to focus on the coefficients of the **intercept** (the constant, they can be used interchangeably) first. This component expresses the biases of our model.(b0, b1 etc.) For each variable, it is the measure of how the change in that variable affects the independent one.

**The standard error**, indicates the accuracy of our prediction for each variable. Lower the standard error better the estimate :)

There are also other values called **t statistics and its P-value**. The null hypothesis in this test is H0: B=0 (Is the coefficient equal to zero?) *P-value < 0.05 means that the variable is significant. 0.000 shows us size is a significant variable when predicting house prices!

As can be clearly seen from the explanations I have made so far, our model needs improvements. Let's start working on it!

# Dealing with missing values

There is no NaN value in our data set, but it would be good to check 0's or 999's for the sake of data consistency. I don't see any irrelevant max values in the describe output but there are some zeros to check.

We have to deal with 49 houses that look free, plus 2 houses each without a bedroom and a bathroom. Since they are few in number, we can remove these values directly from our data set, but since I want to try a few different methods, I will try to keep them by replacing them with appropriate values.

I will leave waterfront as it is because zero values indicate there is no "waterfront" rather than the information being unknown. Same for: view, sqft_basement and yr_renovated. 


In [ ]:
raw_data.isnull().sum()

In [ ]:
raw_data[raw_data==0].count()

In [ ]:
zero_price = raw_data[raw_data['price']==0]
zero_price.describe().T

In [ ]:
# In order to decide what to do with the null values in the price, 
# it is helpful to know the corelations between features.

corr_matrix = raw_data.corr()

fig, ax = plt.subplots(figsize = (15,6))
sns.heatmap(corr_matrix, annot = True)

In [ ]:
# By looking at the median value of the sqft_living variable, which affects the price the most,
# I divided the prices that appear 0 into two groups. 
# Afterwards, I decided on the value that I would assign to the empty prices in these two groups 
# by looking at the median values of the 3 variables that most affected the price value.

low_price_data = raw_data[(raw_data['sqft_living'] < zero_price['sqft_living'].median()) &
         (raw_data['bathrooms'] < zero_price['bathrooms'].median()) &
         (raw_data['sqft_above'] < zero_price['sqft_above'].median()) ]
low_price = low_price_data.price.median()

high_price_data = raw_data[(raw_data['sqft_living'] > zero_price['sqft_living'].median()) &
         (raw_data['bathrooms'] > zero_price['bathrooms'].median()) &
         (raw_data['sqft_above'] > zero_price['sqft_above'].median()) ]
high_price = high_price_data.price.median()

data_prc = raw_data.copy()
data_prc['price'] = np.where(((data_prc['price']==0) & (data_prc['sqft_living'] > zero_price['sqft_living'].median())), high_price, data_prc.price) 
data_prc['price'] = np.where(((data_prc['price']==0) & (data_prc['sqft_living'] <= zero_price['sqft_living'].median())), low_price, data_prc.price)

data_prc.price[data_prc.price==0].count()

In [ ]:
# I will print the distrubution plots to decide 
# which method to use fill in the unknown zero values in the bedrooms and batromms columns.
# As you may notice, there is some skewness that will affect the mean of both features. 
# I will use the median imputation for replacing zero values.

fig, ax = plt.subplots(1,2, figsize = (20,6))

sns.distplot(ax = ax[0], x= data_prc.bedrooms, color='darkmagenta')
ax[0].set_title('Bedrooms', size = 18)
sns.distplot(ax = ax[1], x = data_prc.bathrooms, color='darkmagenta')
ax[1].set_title('Bathrooms', size = 18)

In [ ]:
data_prc.groupby('bedrooms')[['bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']].mean()

In [ ]:
data_prc['bedrooms'] = data_prc['bedrooms'].replace(0, np.NaN)
data_prc['bedrooms'] = data_prc['bedrooms'].fillna(data_prc.bedrooms.median())

data_prc.bedrooms[data_prc.bedrooms==0].count()

In [ ]:
data_prc['bathrooms'].replace(to_replace = 0, value = data_prc.bathrooms.median(), inplace= True)

data_prc.bathrooms[data_prc.bathrooms==0].count()

In [ ]:
data_prc.groupby('bedrooms')[['bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']].mean()

# Dealing with outliers

One of the most important parts of data preprocessing is detecting and treating the outliers as they affect our analysis and training process in a bad way.

Simply, an outlier is an observation that lies far from the rest of the observations in a given dataset and they cause a skew. We can use some techniques such as boxplots, Z-score or Inter Quantile Range(IQR) to detect outliers. 

After determining these values, the next step is to decide what to do with these values. We can remove these values, use either method quarter-based flooring and covering, or mean/median imputation.

In [ ]:
# A great step in the data exploration is to display the boxplot to see outliers

sns.catplot(x='price', data=data_prc, kind='box', height=3, aspect=3)

In [ ]:
# Also we can print the probability distribution function (PDF) of a variable
# The PDF will show us how that variable is distributed 
# This makes it very easy to spot anomalies, icluded outliers
# The PDF is often the basis on which we decide whether we want to transform a feature

sns.distplot(data_prc.price, color='darkmagenta')

In [ ]:
# I will use the IQR measurement for removing outliers.

Q75 = np.percentile(data_prc['price'],75)
Q25 = np.percentile(data_prc['price'],25)
IQR = Q75-Q25
cutoff = IQR * 1.5
upper = Q75 + cutoff
lower = 1

data1 = data_prc[(data_prc['price']<upper)]



In [ ]:
# What a change!

sns.distplot(data1.price, color='darkmagenta') 

print(data_prc['price'].skew(),',', data1['price'].skew())

In [ ]:
data1.columns.values

In [ ]:
# Let's look other variables.

cols = ['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']

fig, ax = plt.subplots(4,3, figsize = (30,15))

for idx, col in enumerate(cols):
    rn = math.floor(idx/3)
    cn = idx%3
    sns.distplot(ax=ax[rn,cn], x=data1[col], color='darkmagenta')
    ax[rn,cn].set_title(col, size=20)

In [ ]:
# bedrooms:
print(data1.bedrooms.value_counts().sort_index())
sns.catplot(x='bedrooms', y='price', data=data1, height=3, aspect=3)

In [ ]:
# The number of houses with 7,8 and 9 bedrooms seems very low. 
# I will subtract these values

data2 = data1[data1.bedrooms<7]

In [ ]:
# I could not understand the meaning of 0.75 or 1.25 bathroom houses. 
# I'll first convert their types to integers.

data2.bathrooms.value_counts().sort_index()

In [ ]:
data2.bathrooms = data2.bathrooms.astype(int)
print(data2.bathrooms.value_counts().sort_index())

sns.catplot(x='bathrooms', y='price', data=data2, height=3, aspect=3)

In [ ]:
data3 = data2[data2.bathrooms<4]

In [ ]:
q = data3.sqft_living.quantile(0.99)
data4 = data3[data3.sqft_living<q]
print(data3.sqft_living.skew(),',', data4.sqft_living.skew())

In [ ]:
# There is still some skew
# But when we look at the price versus scatter plot, it looks much better now.

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,3))

ax1.scatter(x= 'sqft_living', y= 'price', data = data3, c= 'sqft_living', cmap='inferno')
ax1.set_title('With Outliers')
ax1.set_xlim(0,7000)
ax1.set_ylim(-0.1e6,1.3e6)
ax2.scatter(x= 'sqft_living', y= 'price', data = data4, c= 'sqft_living', cmap='inferno')
ax2.set_title('Without Outliers')
ax2.set_xlim(0,7000)
ax2.set_ylim(-0.1e6,1.3e6)

In [ ]:
q = data4.sqft_lot.quantile(0.99)
data5 = data4[data4.sqft_lot<q]

In [ ]:
# Let's compare the boxplots this time, there is a visible improvement.

fig, ax = plt.subplots(1,2, figsize=(10,6))

sns.boxplot(ax=ax[0], x=data4.sqft_lot, color = 'darkmagenta')
ax[0].set_title('With Outliers')
sns.boxplot(ax=ax[1], x=data5.sqft_lot, color = 'darkmagenta')
ax[1].set_title('Without Outliers')

In [ ]:
data5.floors.value_counts()

In [ ]:
# Again, I will convert the data type to integer.

data6 = data5.copy()
data6.floors = data6.floors.astype(int)
print(data6.floors.value_counts())

sns.catplot(x='floors', y='price', data=data6, height=3, aspect=3)

In [ ]:
# As I said at the beginning, Waterfront is divided into two categories, I will keep it that way.

data6.waterfront.value_counts()

In [ ]:
# Since the number of house with a view is very few, it will be more useful for our analysis to see this feature as 0 and 1. 

print(data6.view.value_counts())

data7=data6.copy()
data7.view = data7.view.map({0:0, 1:1, 2:1, 3:1, 4:1})
print(data7.view.value_counts())



In [ ]:
# There doesn't seem to be many homes in bad shape in the US, I'll just remove 1's.

print(data7.condition.value_counts())

data7 = data7[data7['condition']>1]

In [ ]:
# I will keep sqft_above as is.

data8 = data7.copy()
sns.boxplot(data7.sqft_above, color= 'darkmagenta')

In [ ]:
# I think removing 1% would be enough.

q = data8.sqft_basement.quantile(0.99)
data9 = data8[data8.sqft_basement<q]

In [ ]:
# There are many houses that do not have a basement.

fig, (ax1, ax2) = plt.subplots(1,2, figsize = (10,3))

ax1.scatter(x= 'sqft_basement', y= 'price', data = data8, c= 'sqft_basement', cmap='inferno')
ax1.set_title('With Outliers')
ax1.set_xlim(-100,2500)
ax1.set_ylim(-0.1e6,1.25e6)
ax2.scatter(x= 'sqft_basement', y= 'price', data = data9, c= 'sqft_basement', cmap='inferno')
ax2.set_title('Without Outliers')
ax2.set_xlim(-100,2500)
ax2.set_ylim(-0.1e6,1.25e6)

In [ ]:
# I will keep building year as is.

sns.boxplot(data9.yr_built, color= 'darkmagenta')

In [ ]:
# This is the last feature I will take as renovated(1) not (0).

sns.distplot(data9.yr_renovated, color= 'darkmagenta')

In [ ]:
data9.yr_renovated = pd.np.where(data9.yr_renovated==0,0,1)

In [ ]:
# We've removed most of the outliers. First I'll just continue with numeric values.
# Let's continue by dropping the categorical variables and saving it as a separate data set.

data_pp = data9.drop(['date', 'city', 'street', 'statezip', 'country'], axis=1)
data_pp.describe().T

# Scaling the data

Scaling is transforming our data to fit on a specific scale like 0-100 or 0-1. It is critical when using metrics-based methods of how far away data points are, such as support vector machines (SVM) or k-nearest neighbors (KNN), rather than linear regression. However, I still think it will be useful to scale the data, I will use the Robust Scaler as there is still some outlier in our data.

In [ ]:
data_pp = data_pp.reset_index(drop=True)

In [ ]:
# Define the targets and inputs.

targets = data_pp.iloc[:,:1]
unscaled_inputs = data_pp.drop(['price'], axis = 1)

unscaled_inputs.head()

In [ ]:
# We will divide our data into train and test groups. This is important to avoid overfitting or underfitting.
# Overfitting means, our training has focused on the particular training data set so much, so it has missed the point. 
# Underfitting means the model has not captured the underlying logic of the data.

x_train, x_test, y_train, y_test = train_test_split(unscaled_inputs, targets, test_size=0.2, random_state=42)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [ ]:
# I will keep features that contain only 0 or 1 data separately.

columns_to_scale = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
        'condition', 'sqft_above', 'sqft_basement',
       'yr_built']
columns_to_omit = ['waterfront','view','yr_renovated']
x_train_to_scale = x_train[columns_to_scale]
x_test_to_scale = x_test[columns_to_scale]
x_train_not_to_scale = x_train[columns_to_omit].reset_index(drop=True)
x_test_not_to_scale = x_test[columns_to_omit].reset_index(drop=True)

In [ ]:
# Let's fit the scaler using only the training data, then transform both training and test data.

scaler = RobustScaler()  #StandardScaler()
scaler.fit(x_train_to_scale)

In [ ]:
x_train_scaled = scaler.transform(x_train_to_scale)

In [ ]:
x_test_scaled = scaler.transform(x_test_to_scale)

In [ ]:
# After scaling our training and test data are converted to np.arrays,
# Let's make them pd.DataFrame again and merge them with unscaled features.

x_train_scaled = pd.DataFrame(x_train_scaled, columns=columns_to_scale)
x_train = pd.concat([x_train_scaled,x_train_not_to_scale], axis=1)
y_train = y_train.reset_index(drop=True)

x_test_scaled = pd.DataFrame(x_test_scaled, columns=columns_to_scale)
x_test = pd.concat([x_test_scaled,x_test_not_to_scale], axis=1)
y_test = y_test.reset_index(drop=True)

In [ ]:
# Now, we are ready!

x_train.head(2)

# Model - Linear Regression

In [ ]:
# Let's create the model as we did at the beginning.

x = sm.add_constant(x_train)
results = sm.OLS(y_train, x).fit()

results.summary()

In [ ]:
# We talked about the low contribution of variables with P values above 0.05 to the model, 
# so I'm going to drop the renovation year value and run the model again.

x_train_pv = x_train.drop(['yr_renovated'], axis=1)

In [ ]:
# R2 and Adj. R2 is the same as before. 
# We did well by removing it out, because it's always better to keep the equation simple.

X = sm.add_constant(x_train_pv)
results = sm.OLS(y_train, X).fit()

results.summary()

# Encoding the Categorical Variables

We have developed our model quite well, but I would like to enrich it a bit more by adding some categorical variables. To do this we need to transform categorical data into appropriate numeric values, there is no single method how to approach this issue. Fortunately, python tools of pandas and scikit-learn provide several approaches that can be implemented. 

Here some methods that can be used:

**One-Hot Encoding**
One-Hot Encoding gets each group of any categorical variable a new variable by mapping each group to binary numbers (0 or 1). Used when data is nominal. Newly created binary properties can be considered as dummy variables.<br>
**Dummy Encoding**
The Dummy encoding scheme is similar to one-hot encoding. In the case of one-hot encoding, it uses N binary variables for N categories in a variable. Dummy encoding is a minor improvement over one-hot encoding, it uses N-1 properties to represent N tags/categories.<br>
**Effect Encoding**
When applying this encoding type, the categories are given values in the format -1,0,1. The -1 occurrence is the only difference between One-Hot encoding and effects encoding.<br>
**Label Encoding or Ordinal Encoding**
This type of encoding is used when the variables in the data are ordinal. It converts each label to integer values, and the encoded data represents the label array.

In [ ]:
data_wd = data9.drop(['date', 'street', 'statezip', 'country'], axis=1)

In [ ]:
# Get_dummies is one of the common ways to create dummy variables for categorical features

city_dummies = pd.get_dummies(data_wd.city, drop_first = True)
city_dummies

In [ ]:
# Let's define the variables one more time.

targets = data_wd.iloc[:,:1]

a = data_wd.drop(['price','city'], axis = 1)
unscaled_inputs_wd = pd.concat([a, city_dummies], axis=1)
unscaled_inputs_wd.head(2)

In [ ]:
# Split the targets and inputs into train-test data again.

x_train, x_test, y_train, y_test = train_test_split(unscaled_inputs_wd, targets, test_size=0.2, random_state = 42)

print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

In [ ]:
x_train.columns.values

In [ ]:
columns_to_scale = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'condition', 'sqft_above', 'sqft_basement',
       'yr_built']
columns_to_omit = ['waterfront','view','yr_renovated', 'Auburn', 'Beaux Arts Village',
       'Bellevue', 'Black Diamond', 'Bothell', 'Burien', 'Carnation',
       'Clyde Hill', 'Covington', 'Des Moines', 'Duvall', 'Enumclaw',
       'Fall City', 'Federal Way', 'Inglewood-Finn Hill', 'Issaquah',
       'Kenmore', 'Kent', 'Kirkland', 'Lake Forest Park', 'Maple Valley',
       'Medina', 'Mercer Island', 'Milton', 'Newcastle', 'Normandy Park',
       'North Bend', 'Pacific', 'Preston', 'Ravensdale', 'Redmond',
       'Renton', 'Sammamish', 'SeaTac', 'Seattle', 'Shoreline',
       'Skykomish', 'Snoqualmie', 'Snoqualmie Pass', 'Tukwila', 'Vashon',
       'Woodinville', 'Yarrow Point']
x_train_to_scale = x_train[columns_to_scale]
x_test_to_scale = x_test[columns_to_scale]
x_train_not_to_scale = x_train[columns_to_omit].reset_index(drop=True)
x_test_not_to_scale = x_test[columns_to_omit].reset_index(drop=True)

In [ ]:
# Scale as before

scaler = RobustScaler()

In [ ]:
scaler.fit(x_train_to_scale)

In [ ]:
x_train_scaled = scaler.transform(x_train_to_scale)

In [ ]:
x_test_scaled = scaler.transform(x_test_to_scale)

In [ ]:
x_train_scaled = pd.DataFrame(x_train_scaled, columns=columns_to_scale)
x_train = pd.concat([x_train_scaled,x_train_not_to_scale], axis=1)
y_train = y_train.reset_index(drop=True)

x_test_scaled = pd.DataFrame(x_test_scaled, columns=columns_to_scale)
x_test = pd.concat([x_test_scaled,x_test_not_to_scale], axis=1)
y_test = y_test.reset_index(drop=True)

# Model - Linear Regression

In [ ]:
# Yey! We managed to get much better results now.
# 'sqft_lot' and 'yr_renovated' features seems insignificant when we evaluate them according to their p-values.

X1 = sm.add_constant(x_train)
results = sm.OLS(y_train, X1).fit()

results.summary()

In [ ]:
x_train.columns.values

## Assumption check for Lineer Regression

**1)Linearity** In the name says it all, we need to see a linear relationship between independent and dependent variables. We can basically determine this using scatter plots. It was also important to check for outliers since linear regression is sensitive to outlier effects. To fix: Run a non-linear regression, exponential transformation, log transformation

**2)No Endogeneity:** Omitted variable bias occurs when we forget to include a variable. This is reflected in the error term as the factor you forgot about is included in the error. In this way, the error is not random but includes a systematic part (the omitted variable).

**3)Normality and Homoscedasticy:** We have assumed that the errors have normal distribution, Zero Mean: Having an intercept solves that problem, in real life it is unusual to having that problem. Homoscedasticity: It means to having equal variance. To fix: Look for OVB, Look for outliers, Apply log transformation

**4)No Autocorrelation(no serial correlation):** Durbin Watson score falls between zero and four. <1 and >3 cause an alarm, if we detect the problem, we need to use alternative method.

**5)No Multicollinearity:** Independent variables that effect each others. To Fix= Drop one of the two variables, transform into one new variable.

In [ ]:
# Let's check if there are multicollunearity.

from statsmodels.stats.outliers_influence import variance_inflation_factor

# all features where we want to check for multicollinearity:
variables = x_train[[ 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'condition',  'sqft_basement',
       'yr_built', 'view', 'yr_renovated', 'Auburn', 'Beaux Arts Village',
       'Bellevue', 'Black Diamond', 'Bothell', 'Burien', 'Carnation',
       'Clyde Hill', 'Covington', 'Des Moines', 'Duvall', 'Enumclaw',
       'Fall City', 'Federal Way', 'Inglewood-Finn Hill', 'Issaquah',
       'Kenmore', 'Kent', 'Kirkland', 'Lake Forest Park', 'Maple Valley',
       'Medina', 'Mercer Island', 'Milton', 'Newcastle', 'Normandy Park',
       'North Bend', 'Pacific', 'Preston', 'Ravensdale', 'Redmond',
       'Renton', 'Sammamish', 'SeaTac', 'Seattle', 'Shoreline',
       'Skykomish', 'Snoqualmie', 'Snoqualmie Pass', 'Tukwila', 'Vashon',
       'Woodinville', 'Yarrow Point']]

# we create a new data frame which will include all the VIFs, 
# each variable has its own VIF as this measure is variable specific (not model specific)
vif = pd.DataFrame()

# here we make use of the variance_inflation_factor, which will basically output the respective VIFs 
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]

# Finally, I will include names so it is easier to explore the result
vif["Features"] = variables.columns

vif.sort_values(by='VIF')

In [ ]:
# I will drop 'sqft_above' based on features VIF scores, 
# also 'sqft_lot' and 'yr_renovated' based on p-value that we have already determined.

x_train = x_train.drop(['sqft_above','sqft_lot','yr_renovated'], axis=1)
x_test = x_test.drop(['sqft_above','sqft_lot','yr_renovated'], axis=1)

In [ ]:
# This time let's use sklearn to build our model.

reg = LinearRegression()
reg.fit(x_train, y_train)

In [ ]:
# Let's check the assumption of normality. 
# It seems quite good, right :)

y_hat = reg.predict(x_train)

sns.histplot(y_train - y_hat)

In [ ]:
# To measure adjusted R2, I will write a simple function. (Train)

def adj_R2(x_train,y_train):
  r2 = reg.score(x_train,y_train)
  n = x_train.shape[0]
  p = x_train.shape[1]
  adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
  return adjusted_r2

adj_R2 = adj_R2(x_train,y_train)

In [ ]:
# To measure adjusted R2, I will write a simple function. (Test)

def adj_R2_test(x_test,y_test):
  r2 = reg.score(x_test,y_test)
  n = x_test.shape[0]
  p = x_test.shape[1]
  adjusted_r2 = 1-(1-r2)*(n-1)/(n-p-1)
  return adjusted_r2

adj_R2_test = adj_R2_test(x_test,y_test)

In [ ]:
# Our training and test scores are very similar,
# which means that overfitting was not observed in our model. 
print('R2-Train                      : {0:.2f}'.format(reg.score(x_train, y_train)*100))
print('R2-Test                       : {0:.2f}'.format(reg.score(x_test, y_test)*100))
print('Adj_R2-Train                  : {0:.2f}'.format(adj_R2*100))
print('Adj_R2-Test                   : {0:.2f}'.format(adj_R2_test*100))
print('MSE (Mean Squared Error)      : {0:.0f}'.format(metrics.mean_squared_error(y_train, y_hat)))
print('RMSE (Root Mean Squared Error): {0:.0f}'.format(np.sqrt(metrics.mean_squared_error(y_train, y_hat))))
print('MAE (Mean Ablosute Error)     : {0:.0f}'.format(metrics.mean_absolute_error(y_train, y_hat)))

In [ ]:
# Let's prepare a regression model summary table.

# Weights means for continuous variables: 
# positive weight = shows that as a feature increases in value, so do the price respectively
# negative weight = shows that as a feature increases in value, price decrease

# Weights means for dummy variables:
# positive weight = shows that the respective category(city) is more expensive than the benchmark
# negative weight = shows that the respective category(city) is less expensive than the benchmark 


reg_summary = pd.DataFrame(x_train.columns.values, columns = ['Features'])
reg_summary['Weights'] = reg.coef_.reshape(-1,1)
reg_summary.index = reg_summary.index +1
reg_summary.loc[0] = ['Intercept (b0)', reg.intercept_[0]]
reg_summary = reg_summary.sort_index()
reg_summary


Our model has reached an acceptable point. Our next step should be to analyze our erroneous predictions in detail and work on them to improve our predictions. 💪

I hope that you find it useful. As I will be happy to improve myself, your comments and feedbacks are always welcome, as are suggestions for additional information that could usefully be included. Thank you! 🌸😊

